# Notícias e Tweets Eleições: gerando amostras

In [11]:
import ipy_table
import re
%run ../aux/postgres.py

postgres functions loaded...


In [75]:
# criando amostras
for i in range(11, 14):
    ii = str(i).zfill(2)
    cmd = "INSERT INTO amostra(descricao) VALUES ('Eleições Notícia " + ii + "');"
    print(cmd)
    #execSQL(cmd)
    cmd = "INSERT INTO amostra(descricao) VALUES ('Eleições Notícia " + ii + " [no similar]');"
    print(cmd)
    #execSQL(cmd)

INSERT INTO amostra(descricao) VALUES ('Eleições Notícia 11');
INSERT INTO amostra(descricao) VALUES ('Eleições Notícia 11 [no similar]');
INSERT INTO amostra(descricao) VALUES ('Eleições Notícia 12');
INSERT INTO amostra(descricao) VALUES ('Eleições Notícia 12 [no similar]');
INSERT INTO amostra(descricao) VALUES ('Eleições Notícia 13');
INSERT INTO amostra(descricao) VALUES ('Eleições Notícia 13 [no similar]');


In [ ]:
cmd = "select * from amostra where descricao like 'Eleições Notícia%'"
res = query(cmd)
res

[(104, 'Eleições Notícia 01', None, None, None, None),
 (105, 'Eleições Notícia 01 [no similar]', None, None, None, None),
 (106, 'Eleições Notícia 02', None, None, None, None),
 (107, 'Eleições Notícia 02 [no similar]', None, None, None, None),
 (108, 'Eleições Notícia 03', None, None, None, None),
 (109, 'Eleições Notícia 03 [no similar]', None, None, None, None),
 (110, 'Eleições Notícia 04', None, None, None, None),
 (111, 'Eleições Notícia 04 [no similar]', None, None, None, None),
 (112, 'Eleições Notícia 05', None, None, None, None),
 (113, 'Eleições Notícia 05 [no similar]', None, None, None, None),
 (114, 'Eleições Notícia 06', None, None, None, None),
 (115, 'Eleições Notícia 06 [no similar]', None, None, None, None),
 (116, 'Eleições Notícia 07', None, None, None, None),
 (117, 'Eleições Notícia 07 [no similar]', None, None, None, None),
 (118, 'Eleições Notícia 08', None, None, None, None),
 (119, 'Eleições Notícia 08 [no similar]', None, None, None, None),
 (120, 'Eleições

In [62]:
cmd = ("SELECT observacao, count(*) " +
       "FROM public.analisetweet " +
       "WHERE codanaliseamostra = 121 " +
       "group by 1 " +
       "order by 1")
res = query(cmd)
res

[('not01', 1920),
 ('not02', 9),
 ('not03', 9199),
 ('not04', 2130),
 ('not05', 42),
 ('not06', 181662),
 ('not06;not08', 135),
 ('not06;not10', 2),
 ('not07', 56),
 ('not08', 10580),
 ('not09', 1110),
 ('not10', 478),
 ('not11', 30),
 ('not12', 1),
 ('not13', 1)]

In [17]:
cmd = ("select t.text, am.similaridade " +
        "from tweet t inner join amostratweet am on am.codtweet = t.codtweet " +
       "where am.codamostra = 101 and similaridade > 0.24  limit 100")
tweets = query(cmd)
print(len(tweets))

100


In [18]:
tweets

[('RT @Comprova: 6 | É falsa a inscrição preconceituosa sobre nordestinos em camisa de filho de Bolsonaro | https://t.co/MR9p9jmwvy',
  0.242938),
 ('A atriz global Luma Costa se envolveu em uma polêmica no último fim de semana ao contestar a campanha #EleNão , contrária à candidatura de Jair Bolsonaro (PSL) à presidência. Pelas redes sociais, Luma chegou a dizer que as mulheres participantes do ato são “burras”.  pic.twitter.com/CA4wPbayta',
  0.256293),
 ('A manifestação #EleNão  em repúdio ao candidato a presidente Jair Bolsonaro, que se espalhou por cidades brasileiras neste sábado, foi a maior manifestação de mulheres na história do Brasil. Foi também das maiores manifestações contra um candidato, independentemente das mulheres.',
  0.24505),
 ('Segue o trajeto da CARREATA VOLUNTÁRIA em prol da candidatura do @jairbolsonaro  à Presidência da República que ocorrerá no dia 06/10/2018 às 14 horas em MONTES CLAROS MG. Contamos com sua presença! Nessa reta final é ainda mais importante

In [126]:
def buscaTweets(filtro, cod1, cod2, onlyVerify=False):
    cod1 = str(cod1)
    cod2 = str(cod2)

    
    cmd = "DELETE FROM amostratweet WHERE codamostra = " + cod1
    print(cmd)
    execSQL(cmd)
    
    cmd = "DELETE FROM amostratweet WHERE codamostra = " + cod2
    print(cmd)
    execSQL(cmd)
    print("")

    cmd = ("SELECT COUNT(*) " +
           "FROM amostratweet at " +
           "INNER JOIN tweet t ON t.codtweet = at.codtweet " +
           "WHERE at.codamostra = 130 " +
           filtro
          )
    count = query(cmd)
    print("Total de noticias: %s\n" % count[0])

    cmd = ("SELECT t.text " +
           "FROM amostratweet at " +
           "INNER JOIN tweet t ON t.codtweet = at.codtweet " +
           "WHERE at.codamostra = 130 " +
           filtro +
           "LIMIT 5")
    tws = query(cmd)

    for tw in tws:
        print(tw)
    
    if onlyVerify:
        return
    # gerando amostra

    
    
    #execSQL(cmd)
    try:
        cmd = ("INSERT INTO amostratweet (codamostra, codtweet) " +
               "SELECT " + cod1 + ", t.codtweet " +
               "FROM amostratweet at " +
               "INNER JOIN tweet t ON t.codtweet = at.codtweet " +
               "WHERE at.codamostra = 130 " +
               filtro )
        print(cmd+"\n")
        execSQL(cmd)
    except psycopg2.IntegrityError as e:
        print(e)

    try:
        cmd = ("INSERT INTO amostratweet (codamostra, codtweet) " +
               "SELECT " + cod2 + ", t.codtweet " +
               "FROM amostratweet at " +
               "INNER JOIN tweet t ON t.codtweet = at.codtweet " +
               "WHERE at.codamostra = 130 " +
               filtro )
        print(cmd+"\n")
        execSQL(cmd)    
    except psycopg2.IntegrityError as e:
        print(e)
    
    cmd = "SELECT count(*) FROM amostratweet WHERE codamostra = " + str(cod1)
    print(cmd)
    total = query(cmd)
    print("total %s: %s\n" % (cod1, total[0][0]))

    cmd = "SELECT count(*) FROM amostratweet WHERE codamostra = " + str(cod2)
    print(cmd)
    total = query(cmd)
    print("total %s: %s" % (cod2, total[0][0]))

# Notícia 01:

É falsa a inscrição preconceituosa sobre nordestinos em camisa de filho de Bolsonaro

In [127]:
filtro = "AND lower(t.text) like '%camisa%' AND lower(t.text) like '%nordest%' "
buscaTweets(filtro, 104, 105)

DELETE FROM amostratweet WHERE codamostra = 104
DELETE FROM amostratweet WHERE codamostra = 105

Total de noticias: 4611

('RT @MajiwkiJacques: Não vejo camisas do @Haddad_Fernando, adesivos nos carros, pessoas fazendo manifestações, e olha q sou do Nordeste, com…',)
('RT @MajiwkiJacques: Não vejo camisas do @Haddad_Fernando, adesivos nos carros, pessoas fazendo manifestações, e olha q sou do Nordeste, com…',)
('RT @MajiwkiJacques: Não vejo camisas do @Haddad_Fernando, adesivos nos carros, pessoas fazendo manifestações, e olha q sou do Nordeste, com…',)
('RT @camisanumero7: Vocês sabem qual região do Brasil mais se beneficiará com a eleição de @jairbolsonaro ?\n\nSim, O NORDESTE !!!\n\nRegião com…',)
('RT @camisanumero7: Vocês sabem qual região do Brasil mais se beneficiará com a eleição de @jairbolsonaro ?\n\nSim, O NORDESTE !!!\n\nRegião com…',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 104, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet W

# Notícia 02:

#Verificamos: Bolsonaro não disse que ‘paciente com câncer custa caro para o Estado’ e que vai ‘cortar essa farra’

In [128]:
filtro = "AND lower(t.text) like '%farra%' AND (lower(t.text) like '%câncer%' OR lower(t.text) like '%cancer%') "
buscaTweets(filtro, 106, 107)

DELETE FROM amostratweet WHERE codamostra = 106
DELETE FROM amostratweet WHERE codamostra = 107

Total de noticias: 12

('@efarsas Precisamos cortar a "farra do fake news"... isso virou um câncer.',)
('Bolsonaro não disse que ‘paciente com câncer custa caro para o Estado’ e que vai ‘cortar essa farra’. Leia em:… https://t.co/nWyPHFiBHS',)
('[Agência Lupa] Bolsonaro não disse que ‘paciente com câncer custa caro para o Estado’ e que vai ‘cortar essa farra’ https://t.co/Jsi6FZYcH0',)
('Câncer e Aids são farras?????Onde vamos parar com esse coiso, cada dia mais louco?. AFF!!!!',)
('RT @Amarulacarvalho: Câncer e Aids são farras?????Onde vamos parar com esse coiso, cada dia mais louco?. AFF!!!! https://t.co/fxOYKrlUJV',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 106, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%farra%' AND (lower(t.text) like '%câncer%' OR lower(t.text) like '%cancer%') 

INSERT

# Notícia 03:

Post com propostas atribuídas a Bolsonaro mistura informações falsas e verdadeiras

In [129]:
filtro = (
    "AND lower(t.text) like '%propostas%' "
    "AND lower(t.text) like '%bolsonaro%' "
    "AND lower(t.text) like '%fim%' "
    "AND lower(t.text) like '%maternidade%' "
)
buscaTweets(filtro, 108, 109)

DELETE FROM amostratweet WHERE codamostra = 108
DELETE FROM amostratweet WHERE codamostra = 109

Total de noticias: 9204

('RT @cinefilo_K: PROPOSTAS DE BOLSONARO\n- Fim do 13º e férias\n- Redução da licença maternidade\n- IR igual pra ricos e POBRES\n- Volta da CPMF…',)
('RT @cinefilo_K: PROPOSTAS DE BOLSONARO\n- Fim do 13º e férias\n- Redução da licença maternidade\n- IR igual pra ricos e POBRES\n- Volta da CPMF…',)
('RT @cinefilo_K: PROPOSTAS DE BOLSONARO\n- Fim do 13º e férias\n- Redução da licença maternidade\n- IR igual pra ricos e POBRES\n- Volta da CPMF…',)
('RT @cinefilo_K: PROPOSTAS DE BOLSONARO\n- Fim do 13º e férias\n- Redução da licença maternidade\n- IR igual pra ricos e POBRES\n- Volta da CPMF…',)
('RT @cinefilo_K: PROPOSTAS DE BOLSONARO\n- Fim do 13º e férias\n- Redução da licença maternidade\n- IR igual pra ricos e POBRES\n- Volta da CPMF…',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 108, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = a

# Notícia 04:

General Mourão não propôs o confisco da poupança

In [130]:
filtro = (
    "AND lower(t.text) like '%confisco%' "
    "AND lower(t.text) like '%poupan%' "
)
buscaTweets(filtro, 110, 111)

DELETE FROM amostratweet WHERE codamostra = 110
DELETE FROM amostratweet WHERE codamostra = 111

Total de noticias: 2304

('RT @HenriqueFontana: URGENTE! Mourão, vice do Bolsonaro, também propõe confisco da poupança https://t.co/HIg2N07NLT',)
('RT @folha: Não é verdade que general Mourão, vice de Bolsonaro, propôs o confisco da poupança https://t.co/NJEVZe3Imn',)
('RT @folha: Não é verdade que general Mourão, vice de Bolsonaro, propôs o confisco da poupança https://t.co/NJEVZe3Imn',)
('RT @folha: Não é verdade que general Mourão, vice de Bolsonaro, propôs o confisco da poupança https://t.co/NJEVZe3Imn',)
('RT @folha: Não é verdade que general Mourão, vice de Bolsonaro, propôs o confisco da poupança https://t.co/NJEVZe3Imn',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 110, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%confisco%' AND lower(t.text) like '%poupan%' 

INSERT INTO amostratweet (co

## Notícia 05:

Áudio atribuído a Jair Bolsonaro no hospital é falso

In [131]:
filtro = (
    "AND lower(t.text) like '%udio%' "
    "AND lower(t.text) like '%hospital%' "
)
buscaTweets(filtro, 112, 113)

DELETE FROM amostratweet WHERE codamostra = 112
DELETE FROM amostratweet WHERE codamostra = 113

Total de noticias: 283

('RT @Profdrclaudio: O Povo baiano é simpático, hospitaleiro, alegre e têm musicalidade sem igual.São honestos e trabalhadores. Este lixo cha…',)
('RT @Profdrclaudio: O Povo baiano é simpático, hospitaleiro, alegre e têm musicalidade sem igual.São honestos e trabalhadores. Este lixo cha…',)
('RT @Profdrclaudio: O Povo baiano é simpático, hospitaleiro, alegre e têm musicalidade sem igual.São honestos e trabalhadores. Este lixo cha…',)
('RT @Profdrclaudio: O Povo baiano é simpático, hospitaleiro, alegre e têm musicalidade sem igual.São honestos e trabalhadores. Este lixo cha…',)
('RT @Profdrclaudio: O Povo baiano é simpático, hospitaleiro, alegre e têm musicalidade sem igual.São honestos e trabalhadores. Este lixo cha…',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 112, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.c

# Notícia 06:

#Verificamos: Imagem apresentada como ‘capa do kit gay’ não está no material

In [132]:
filtro = (
    "AND lower(t.text) like '%kit%' "
    "AND lower(t.text) like '%gay%' "
)
buscaTweets(filtro, 114, 115)

DELETE FROM amostratweet WHERE codamostra = 114
DELETE FROM amostratweet WHERE codamostra = 115

Total de noticias: 285142

('RT @QuebrandoOTabu: Tá circulando vídeo numa livraria que mostra que o livro que Bolsonaro disse ser kit gay existe.\n\nNinguém disse que o l…',)
('RT @louieponto: ah pronto bolsonaro falando do kit gay no programa eleitoral depois de ter sido proibido pelo TSE porque é UMA MENTIRA',)
('RT @vaIehomossexuaI: Bolsonaro meteu o vídeo do secretário do Haddad falando do beijo lésbico no lance do kit gay, hoje as tia do zap vão f…',)
('RT @vaIehomossexuaI: Bolsonaro meteu o vídeo do secretário do Haddad falando do beijo lésbico no lance do kit gay, hoje as tia do zap vão f…',)
('RT @rogeriotomazjr: O sinais da #HoraDaVirada que o Bolsonaro sentiu e entrou em desespero:\n- uso do kitgay no horário eleitoral mesmo proi…',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 114, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.c

# Notícia 07:

Projeto não torna a pedofilia um ato legal e nem tem participação de Haddad

In [133]:
filtro = (
    "AND lower(t.text) like '%pedofilia%' "
    "AND ( lower(t.text) like '%projeto%' OR lower(t.text) like '%236%' )"
)
buscaTweets(filtro, 116, 117)

DELETE FROM amostratweet WHERE codamostra = 116
DELETE FROM amostratweet WHERE codamostra = 117

Total de noticias: 1682

('RT @estadaoverifica: Projeto não torna a pedofilia um ato legal e nem tem participação de Haddad #Comprova https://t.co/u6TwZX8TYt',)
('RT @UOL: Fotografia circula em redes sociais | Projeto não torna pedofilia um ato legal nem tem a participação de Haddad https://t.co/wwd70…',)
('RT @GauchaZH: O projeto de lei em questão nada tem a ver com legalização da pedofilia e tramita no Congresso Nacional sob relatoria de um s…',)
('RT @guilouress: Eu entro no facebook e tá la as fake news sobre existir projeto de lei do Pt a favor da pedofilia. Mano, se esses Cristãos…',)
('RT @UOLNoticias: Projeto não torna a pedofilia um ato legal nem tem participação de Haddad #comprova  https://t.co/bddz58GQ54',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 116, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower

# Notícia 08:

"Mamadeiras eróticas" não foram distribuídas em creches pelo PT

In [134]:
filtro = (
    "AND lower(t.text) like '%mamadeira%' "
)
buscaTweets(filtro, 118, 119)

DELETE FROM amostratweet WHERE codamostra = 118
DELETE FROM amostratweet WHERE codamostra = 119

Total de noticias: 15426

('RT @heisenboff: no passado: "em 2018 teremos carros voadores"\n\nem 2018: tendo que desmentir fake news que diz que distribuíram mamadeira co…',)
('RT @AbalFelipe: Os caras acreditam em mamadeira com bico de pênis, que o PT pagou 90 milhões pro Roger Waters e que o Irã tem planos pra in…',)
('RT @heisenboff: no passado: "em 2018 teremos carros voadores"\n\nem 2018: tendo que desmentir fake news que diz que distribuíram mamadeira co…',)
('@Cellymonteiro @Tatiana76928697 Já vi que vc acredita em tudo que o Haddad diz.\nO lance da mamadeira pênis foi cois… https://t.co/tcBBpGTH0Q',)
('essa mamadeira de piroca com certeza é a melhor fake news de TODAS',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 118, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%mamadeira%' 

INSERT INTO 

# Notícia 09:

Vídeo em que Haddad diz que eleição acabou é de 2016

In [135]:
filtro = (
    "AND lower(t.text) like '%jogou%' "
    "AND lower(t.text) like '%toalha%' "
)
buscaTweets(filtro, 120, 121)

DELETE FROM amostratweet WHERE codamostra = 120
DELETE FROM amostratweet WHERE codamostra = 121

Total de noticias: 1843

('RT @cidapitanga: Folha reconhece que foi incapaz de olhar seus próprios leitores.\nJOGOU A TOALHA \n\nPRIMEIRO TURNO GENTE!!\n\n"A Folha de S. P…',)
('RT @Dom_Rezende33: Até Ibope já jogou a toalha\n\nVotos válidos:\nBolsonaro 59% x 41% Haddad https://t.co/64IdlpdDS4',)
('RT @Dom_Rezende33: Até Ibope já jogou a toalha\n\nVotos válidos:\nBolsonaro 59% x 41% Haddad https://t.co/64IdlpdDS4',)
('RT @Dom_Rezende33: Até Ibope já jogou a toalha\n\nVotos válidos:\nBolsonaro 59% x 41% Haddad https://t.co/64IdlpdDS4',)
('RT @Dom_Rezende33: Até Ibope já jogou a toalha\n\nVotos válidos:\nBolsonaro 59% x 41% Haddad https://t.co/64IdlpdDS4',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 120, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%jogou%' AND lower(t.text) like '%toalha%' 

INSER

# Notícia 10:

"Ao completar 5 anos de idade, a criança passa a ser propriedade do Estado! Cabe a nós decidir se
menino será menina e vice-versa! Aos pais cabe acatar nossa decisão respeitosamente! Sabemos
o que é melhor para as crianças!"

In [136]:
filtro = (
    "AND lower(t.text) like '%crian%' "
    "AND lower(t.text) like '%propriedade%' "
    "AND lower(t.text) like '%estado%' "
)
buscaTweets(filtro, 122, 123)

DELETE FROM amostratweet WHERE codamostra = 122
DELETE FROM amostratweet WHERE codamostra = 123

Total de noticias: 5410

('RT @AngelJones_1330: 1. Fake News! \n\n- É #FAKE que Haddad disse que criança vira propriedade do Estado aos 5 anos e pode ter seu gênero esc…',)
('RT @portadosfundos: "Não recebeu por WhatsApp? Toda criança a partir de 5 anos agora é propriedade do Estado."\n\nVÍDEO NOVO | Fake News: htt…',)
('RT @portadosfundos: "Não recebeu por WhatsApp? Toda criança a partir de 5 anos agora é propriedade do Estado."\n\nVÍDEO NOVO | Fake News: htt…',)
('RT @portadosfundos: "Não recebeu por WhatsApp? Toda criança a partir de 5 anos agora é propriedade do Estado."\n\nVÍDEO NOVO | Fake News: htt…',)
('RT @portadosfundos: "Não recebeu por WhatsApp? Toda criança a partir de 5 anos agora é propriedade do Estado."\n\nVÍDEO NOVO | Fake News: htt…',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 122, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet

# Notícia 11:

Patrícia Pillar desmente meme e diz que nunca sofreu agressão

In [137]:
filtro = (
    "AND lower(t.text) like '%pillar%' "
    "AND (lower(t.text) like '%batia%' "
    "OR lower(t.text) like '%agress%') "
)
buscaTweets(filtro, 124, 125)

DELETE FROM amostratweet WHERE codamostra = 124
DELETE FROM amostratweet WHERE codamostra = 125

Total de noticias: 35

('RT @karinamarigold: @Haddad_Fernando Não apenas contra vc Haddad, aquela história que o @cirogomes batia na Patrícia Pillar tbm foi fruto d…',)
('@cirogomes Além do escândalo da agressão a Patricia Pillar e mandar prender um jornalista que te fez uma pergunta,… https://t.co/BdxXUJ0z82',)
('RT @eliaxjr: alguns vídeos bônus: Patrícia Pillar desmentindo a fake news de que Ciro batia nela https://t.co/GuPU3LFYMw',)
('@alentringer @KomatsuTekila @wallyssonrg @jairbolsonaro A Patricia Pillar nega a agressão: Verdade. Não agrediu\n\nEx… https://t.co/KMkJo2MCOb',)
('segunda vez que leio hoje essa história de que o ciro batia na patricia pillar. gente, isso é fake!!!! além de já t… https://t.co/o5TS404Hrs',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 124, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND 

# Notícia 12:

Não é verdade que marido de Marina Silva é 'um dos maiores desmatadores da Amazônia'

In [138]:
filtro = (
    "AND lower(t.text) like '%desmatador%' "
    "AND lower(t.text) like '%marina%' "
)
buscaTweets(filtro, 126, 127)

DELETE FROM amostratweet WHERE codamostra = 126
DELETE FROM amostratweet WHERE codamostra = 127

Total de noticias: 5

('Marina, casada com um dos maiores desmatadores da Amazônia ( como Ministra do meio ambiente, consegue as licenças),… https://t.co/D8yabTXjrF',)
('QDO MARINA MOGNO AGIU EM FAVOR DO MARIDO DELA, GRANDE GRILEIRO E DESMATADOR, NINGUÉM SE PREOCUPOU?',)
('RT @RosaACL: QDO MARINA MOGNO AGIU EM FAVOR DO MARIDO DELA, GRANDE GRILEIRO E DESMATADOR, NINGUÉM SE PREOCUPOU? https://t.co/jfdZGwKBQe',)
('RT @RosaACL: QDO MARINA MOGNO AGIU EM FAVOR DO MARIDO DELA, GRANDE GRILEIRO E DESMATADOR, NINGUÉM SE PREOCUPOU? https://t.co/jfdZGwKBQe',)
('@andreiarosaaro @bollemdb Marina (casada com o maior desmatador da amazônia e diz proteger o meio ambiente). FHC um… https://t.co/c0AMnmo8AN',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 126, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%desmatador%' 

# Notícia 13:

Ciro Gomes não garantiu que vai diminuir salários de médicos; site distorce informações

In [139]:
filtro = (
    "AND lower(t.text) like '%sal%rio%' "
    "AND lower(t.text) like '%ciro%' "
    "AND ( lower(t.text) like '%médico%' "
    "OR lower(t.text) like '%medico%' )"
)
buscaTweets(filtro, 128, 129)

DELETE FROM amostratweet WHERE codamostra = 128
DELETE FROM amostratweet WHERE codamostra = 129

Total de noticias: 1

('Acabei de ver Ciro Gomes prometeu baixar os salários dos médicos caso seja eleito? - Clique para ver também ☛… https://t.co/jBRwAftvkk',)
INSERT INTO amostratweet (codamostra, codtweet) SELECT 128, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%sal%rio%' AND lower(t.text) like '%ciro%' AND ( lower(t.text) like '%médico%' OR lower(t.text) like '%medico%' )

INSERT INTO amostratweet (codamostra, codtweet) SELECT 129, t.codtweet FROM amostratweet at INNER JOIN tweet t ON t.codtweet = at.codtweet WHERE at.codamostra = 130 AND lower(t.text) like '%sal%rio%' AND lower(t.text) like '%ciro%' AND ( lower(t.text) like '%médico%' OR lower(t.text) like '%medico%' )

SELECT count(*) FROM amostratweet WHERE codamostra = 128
total 128: 1

SELECT count(*) FROM amostratweet WHERE codamostra = 129
total

# Totais:

In [86]:
cmd = ("SELECT observacao, count(*) " +
       "FROM public.analisetweet " +
       "WHERE codanaliseamostra = 121 " +
       "group by 1 " +
       "order by 1")
res = query(cmd)
res

[('not01', 1920),
 ('not02', 9),
 ('not03', 9199),
 ('not04', 2130),
 ('not05', 267),
 ('not06', 181662),
 ('not06;not08', 135),
 ('not06;not10', 2),
 ('not08', 10580),
 ('not09', 1110),
 ('not10', 478),
 ('not11', 30),
 ('not12', 1),
 ('not13', 1)]

In [140]:
cmd = ("select am.codamostra, am.descricao, count(amt.*) "
       "from amostratweet amt "
       "inner join amostra am ON amt.codamostra = am.codamostra "
       "where amt.codamostra in (select codamostra from amostra where descricao like 'Eleições Notícia%') "
       "group by 1, 2"
       "order by am.descricao"
      )
res = query(cmd)
res

[(104, 'Eleições Notícia 01', 4611),
 (105, 'Eleições Notícia 01 [no similar]', 4611),
 (106, 'Eleições Notícia 02', 12),
 (107, 'Eleições Notícia 02 [no similar]', 12),
 (108, 'Eleições Notícia 03', 9204),
 (109, 'Eleições Notícia 03 [no similar]', 9204),
 (110, 'Eleições Notícia 04', 2304),
 (111, 'Eleições Notícia 04 [no similar]', 2304),
 (112, 'Eleições Notícia 05', 283),
 (113, 'Eleições Notícia 05 [no similar]', 283),
 (114, 'Eleições Notícia 06', 285142),
 (115, 'Eleições Notícia 06 [no similar]', 285142),
 (116, 'Eleições Notícia 07', 1682),
 (117, 'Eleições Notícia 07 [no similar]', 1682),
 (118, 'Eleições Notícia 08', 15426),
 (119, 'Eleições Notícia 08 [no similar]', 15426),
 (120, 'Eleições Notícia 09', 1843),
 (121, 'Eleições Notícia 09 [no similar]', 1843),
 (122, 'Eleições Notícia 10', 5410),
 (123, 'Eleições Notícia 10 [no similar]', 5410),
 (124, 'Eleições Notícia 11', 35),
 (125, 'Eleições Notícia 11 [no similar]', 35),
 (126, 'Eleições Notícia 12', 5),
 (127, 'Eleiç

In [92]:
cmd = ("select am.codamostra, am.descricao, count(amt.*) "
       "from amostratweet amt "
       "inner join amostra am ON amt.codamostra = am.codamostra "
       "where amt.codamostra in (select codamostra from amostra where descricao like 'Eleições Notícia%') "
       "group by 1, 2"
       "order by am.descricao"
      )
res = query(cmd)
res

[(104, 'Eleições Notícia 01', 2291),
 (105, 'Eleições Notícia 01 [no similar]', 168),
 (106, 'Eleições Notícia 02', 10),
 (107, 'Eleições Notícia 02 [no similar]', 10),
 (108, 'Eleições Notícia 03', 9201),
 (109, 'Eleições Notícia 03 [no similar]', 9201),
 (110, 'Eleições Notícia 04', 2130),
 (111, 'Eleições Notícia 04 [no similar]', 2130),
 (112, 'Eleições Notícia 05', 267),
 (113, 'Eleições Notícia 05 [no similar]', 267),
 (114, 'Eleições Notícia 06', 181917),
 (115, 'Eleições Notícia 06 [no similar]', 181917),
 (116, 'Eleições Notícia 07', 1650),
 (117, 'Eleições Notícia 07 [no similar]', 1650),
 (118, 'Eleições Notícia 08', 10715),
 (119, 'Eleições Notícia 08 [no similar]', 10715),
 (120, 'Eleições Notícia 09', 1110),
 (121, 'Eleições Notícia 09 [no similar]', 1110),
 (122, 'Eleições Notícia 10', 480),
 (123, 'Eleições Notícia 10 [no similar]', 480),
 (124, 'Eleições Notícia 11', 30),
 (125, 'Eleições Notícia 11 [no similar]', 30),
 (126, 'Eleições Notícia 12', 1),
 (127, 'Eleições

In [142]:
cmd = ("select am.codamostra, am.descricao, count(amt.*) "
       "from amostratweet amt "
       "inner join amostra am ON amt.codamostra = am.codamostra "
       "where amt.codamostra in (select codamostra from amostra where descricao like 'Eleições Notícia%similar]') "
       "group by 1, 2"
       "order by am.descricao"
      )
res = query(cmd)
res

[(105, 'Eleições Notícia 01 [no similar]', 229),
 (107, 'Eleições Notícia 02 [no similar]', 8),
 (109, 'Eleições Notícia 03 [no similar]', 24),
 (111, 'Eleições Notícia 04 [no similar]', 210),
 (113, 'Eleições Notícia 05 [no similar]', 46),
 (115, 'Eleições Notícia 06 [no similar]', 269129),
 (117, 'Eleições Notícia 07 [no similar]', 43),
 (119, 'Eleições Notícia 08 [no similar]', 6973),
 (121, 'Eleições Notícia 09 [no similar]', 347),
 (123, 'Eleições Notícia 10 [no similar]', 42),
 (125, 'Eleições Notícia 11 [no similar]', 23),
 (127, 'Eleições Notícia 12 [no similar]', 3),
 (129, 'Eleições Notícia 13 [no similar]', 1)]

In [141]:
cmd = ("select am.codamostra, am.descricao, count(amt.*) "
       "from amostratweet amt "
       "inner join amostra am ON amt.codamostra = am.codamostra "
       "where amt.codamostra in (select codamostra from amostra where descricao like 'Eleições Notícia%similar]') "
       "group by 1, 2"
       "order by am.descricao"
      )
res = query(cmd)
res

[(105, 'Eleições Notícia 01 [no similar]', 4611),
 (107, 'Eleições Notícia 02 [no similar]', 12),
 (109, 'Eleições Notícia 03 [no similar]', 9204),
 (111, 'Eleições Notícia 04 [no similar]', 2304),
 (113, 'Eleições Notícia 05 [no similar]', 283),
 (115, 'Eleições Notícia 06 [no similar]', 285142),
 (117, 'Eleições Notícia 07 [no similar]', 1682),
 (119, 'Eleições Notícia 08 [no similar]', 15426),
 (121, 'Eleições Notícia 09 [no similar]', 1843),
 (123, 'Eleições Notícia 10 [no similar]', 5410),
 (125, 'Eleições Notícia 11 [no similar]', 35),
 (127, 'Eleições Notícia 12 [no similar]', 5),
 (129, 'Eleições Notícia 13 [no similar]', 1)]

In [102]:
cmd = ("select am.descricao, count(amt.*) "
       "from amostratweet amt "
       "inner join amostra am ON amt.codamostra = am.codamostra "
       "where amt.codamostra in ("
       "select codamostra from amostra "
       "where descricao like 'Eleições Notícia%' and descricao not like '%similar%' ) "
       "group by 1"
       "order by am.descricao"
      )
res = query(cmd)
res

[('Eleições Notícia 01', 2291),
 ('Eleições Notícia 02', 10),
 ('Eleições Notícia 03', 9201),
 ('Eleições Notícia 04', 2130),
 ('Eleições Notícia 05', 267),
 ('Eleições Notícia 06', 181917),
 ('Eleições Notícia 07', 1650),
 ('Eleições Notícia 08', 10715),
 ('Eleições Notícia 09', 1110),
 ('Eleições Notícia 10', 480),
 ('Eleições Notícia 11', 30),
 ('Eleições Notícia 12', 1),
 ('Eleições Notícia 13', 1)]